In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
data = pd.read_csv("balance.csv")
data.columns = data.iloc[3,:]
data = data.drop(data.index[[0,1,2,3]]).reset_index(drop=True)

## 总收入

In [3]:
# 总收入
zongshouru = data["收入金额（+元）"].astype("float").sum() *100 / 100
zongshouru

213025.73

In [4]:
# 交易收款-交易收款
new_data = data.dropna(subset=['业务描述'])
jiaoyishoukuan = new_data[new_data["业务描述"].str.contains("交易收款")]["收入金额（+元）"].astype("float").sum()
jiaoyishoukuan

193024.53

In [5]:
# 商家红包退回
new2 = data.dropna(subset=["备注"])
hongbaotuihui = new2[new2["备注"].str.contains("红包退回")]["收入金额（+元）"].astype("float").sum()
hongbaotuihui

20001.2

## 总支出

In [6]:
# 总支出
zongzhichu = data["支出金额（-元）"].astype("float").sum() *100/100
zongzhichu

-17305.62

In [7]:
# 网红佣金的支出
new_data = data.dropna(subset=['业务描述'])
whyongjin = new2[new2["备注"].str.contains("网红佣金")]["支出金额（-元）"].astype("float").sum()
whyongjin

-4952.699999999999

In [8]:
# 软件服务费-服务费-分期
fenqi = new_data[new_data["业务描述"].str.contains("分期")]["支出金额（-元）"].astype("float").sum()
fenqi

-2979.05

In [9]:
# 返点积分
fandianjifen = new_data[new_data["业务描述"].str.contains("返点积分")]["支出金额（-元）"].astype("float").sum()
fandianjifen

-1063.3200000000002

In [10]:
# 软件服务费-服务费
fuwufei = new2[new2["备注"].str.contains("支付服务费")]["支出金额（-元）"].astype("float").sum()
fuwufei

-786.51

In [11]:
# 软件服务费-佣金
pingtaiyongjin = new_data[new_data["业务描述"].str.contains("软件服务费") & new_data["业务描述"].str.contains("佣金")]["支出金额（-元）"].astype("float").sum()
pingtaiyongjin

-7524.04

In [12]:
whyongjin + fenqi + fandianjifen + fuwufei + pingtaiyongjin

-17305.62

## 核对

#### 收入

In [76]:
service = pd.read_excel("service_fee.xlsx")

In [18]:
service.to_csv("service.csv", header = True)

In [38]:
service = pd.read_csv("service.csv")
data = pd.read_csv("balance.csv")
data.columns = data.iloc[3,:]
data = data.drop(data.index[[0,1,2,3]]).reset_index(drop=True)

In [41]:
dingdan_id = service["父订单编号"].values.astype(str)
data["业务基础订单号"] = data["业务基础订单号"].apply(lambda x: x[:-1])
data = data.set_index('业务基础订单号')

In [42]:
store = defaultdict(int)
data["收入金额（+元）"] = data["收入金额（+元）"].astype(float)
for i in dingdan_id:
    try:
        temp = data.loc[i,:]
        temp_shouru = temp["收入金额（+元）"].sum()
        store[i] = temp_shouru
    except KeyError:
        store[i] = "Do not exist"

In [43]:
check_shouru = defaultdict(str)
service["父订单编号"] = service["父订单编号"].astype(str)
new_service = service.set_index("父订单编号")
for i in store.keys():
    service_fee_temp = new_service.loc[i, :]["实际成交价格"]
    check_shouru[i] = (store[i] == service_fee_temp)
    

In [105]:
check_shouru

defaultdict(str,
            {'1076954400743072025': False,
             '596129828744860799': True,
             '1073116000093076875': True,
             '1073005408984068487': True,
             '1084321153766744786': True,
             '1073527010421679883': True,
             '1086388417778773009': 父订单编号
             1086388417778773009    False
             1086388417778773009    False
             1086388417778773009    False
             Name: 实际成交价格, dtype: bool,
             '1078061025987387152': True,
             '596595333433349696': True,
             '605833996150790402': True,
             '1077364768438382988': True,
             '1077869473737382988': True,
             '1073838019192885021': True,
             '1076351522963320530': True,
             '1078616611464948781': True,
             '1074386561412117350': True,
             '1075816673369312283': True,
             '1076067779825312283': True,
             '1075856737417312283': True,
             '1073492

#### 网红佣金

In [101]:
service = pd.read_csv("service.csv")
data = pd.read_csv("balance.csv")
data.columns = data.iloc[3,:]
data = data.drop(data.index[[0,1,2,3]]).reset_index(drop=True)

In [102]:
data = data.dropna(subset=["备注"])
temp_data = data[data["备注"].str.contains("网红佣金代扣")]
temp_data["id"] = data["备注"].apply(lambda x: x[8:-1])
temp_data = temp_data.set_index("id")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [103]:
check_wanghong = defaultdict(int)
count = 0
service["子订单编号"] = service["子订单编号"].astype(str)
zidingdan = service["子订单编号"].values
new_service = service.set_index("子订单编号")
for i in zidingdan:
    count+=1
    try:
        temp1 = -new_service.loc[i,"佣金"]
        temp2 = float(temp_data.loc[i, "支出金额（-元）"])
        check_wanghong[i] = (temp1 == temp2)
    except KeyError:
        check_wanghong[i] = False

In [104]:
check_wanghong

defaultdict(int,
            {'1076954400743072025': True,
             '596129828744860799': True,
             '1073116000093076875': True,
             '1073005408984068487': True,
             '1084321153766744786': True,
             '1073527010421679883': True,
             '1086388417781773009': 子订单编号
             1086388417781773009    True
             1086388417781773009    True
             Name: 佣金, dtype: bool,
             '1078061025987387152': True,
             '596595333433349696': False,
             '605833996150790402': True,
             '1077364768438382988': True,
             '1077869473737382988': True,
             '1073838019192885021': True,
             '1076351522964320530': True,
             '1078616611464948781': False,
             '1074386561412117350': True,
             '1075816673369312283': True,
             '1076067779825312283': True,
             '1075856737417312283': True,
             '1073492961853627734': False,
             '10723229798

1073492961853627734: not found
